# 데이터 전처리

In [84]:
from PIL import Image
import os, glob

image_dir_path = os.getenv("HOME") + "/mini_projects/rps/train/scissor"
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  


target_size=(28,28) 
# 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("가위 이미지 resize 완료!")


image_dir_path = os.getenv("HOME") + "/mini_projects/rps/train/rock"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  


target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("바위 이미지 resize 완료!")


image_dir_path = os.getenv("HOME") + "/mini_projects/rps/train/paper"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  


target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("보 이미지 resize 완료!")

이미지 디렉토리 경로:  /home/aiffel/mini_projects/rps/train/scissor
가위 이미지 resize 완료!
이미지 디렉토리 경로:  /home/aiffel/mini_projects/rps/train/rock
바위 이미지 resize 완료!
이미지 디렉토리 경로:  /home/aiffel/mini_projects/rps/train/paper
보 이미지 resize 완료!


# 훈련 데이터 적재

### 루브릭 평가문항 2. 오버피팅을 극복하기 위한 적절한 시도가 있었는가?

0. 훈련 데이터셋에 소요된 총 이미지 개수 5702 장  / 테스트 데이터 셋에 소요된 총 이미지 개수 643장
1. 훈련 데이터셋과 테스트 데이터셋의 구분
2. 테스트 데이터셋의 이미지들은 훈련 데이터셋 있는 이미지들과 최대한 다른 이미지들로 선별하여 분류
3. 데이터 normalization 수행

In [85]:
import numpy as np

def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data= 5702  
    img_size=28
    color=3 
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/mini_projects/rps/train/"   

(x_train, y_train)=load_data(image_dir_path)    # 핵심 함수 : 훈련데이터의 입력과 레이블 생성하는 함수!!

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 5698 입니다.
x_train shape: (5702, 28, 28, 3)
y_train shape: (5702,)


## 훈련 데이터 normalization

In [86]:
print('최소값:',np.min(x_train), ' 최대값:',np.max(x_train))

x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

최소값: 0  최대값: 255


## 모델 설계

In [87]:
from tensorflow import keras

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,3)))
# Conv2D(16) 얼마나 다양한 이미지의 특징을 살펴볼 것인가? (입력 이미지가 다양할 수록 더 많은 특징 고려 필요)
# input_shape=(28,28,3) >> 입력 이미지의 형태 3 : 채널 정보 컬러사진 RGB니까 3
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
# Conv2D(32) 얼마나 다양한 이미지의 특징을 살펴볼 것인가? (입력 이미지가 다양할 수록 더 많은 특징 고려 필요)
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
# Dense(32) : 분류기 알고리즘을 얼마나 복잡하게 할 것인가? (복잡한 문제일 수록 이 숫자를 늘린다) 
model.add(keras.layers.Dense(3, activation='softmax'))
# 최종 분류기의 클래스 숫자 3 : 가위, 바위, 보

model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 800)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 32)                25632     
_________________________________________________________________
dense_41 (Dense)             (None, 3)               

# 모델 훈련 (훈련 데이터 셋 학습)

In [88]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# 모델 훈련
model.fit(x_train_norm, y_train, epochs=10)

Epoch 1/10
179/179 [==============================] - 4s 20ms/step - loss: 0.9591 - accuracy: 0.5284
Epoch 2/10
179/179 [==============================] - 0s 2ms/step - loss: 0.6190 - accuracy: 0.7524
Epoch 3/10
179/179 [==============================] - 0s 2ms/step - loss: 0.4289 - accuracy: 0.8430
Epoch 4/10
179/179 [==============================] - 0s 2ms/step - loss: 0.3070 - accuracy: 0.8923
Epoch 5/10
179/179 [==============================] - 0s 2ms/step - loss: 0.2322 - accuracy: 0.9225
Epoch 6/10
179/179 [==============================] - 0s 2ms/step - loss: 0.1664 - accuracy: 0.9476
Epoch 7/10
179/179 [==============================] - 0s 2ms/step - loss: 0.1340 - accuracy: 0.9593
Epoch 8/10
179/179 [==============================] - 0s 2ms/step - loss: 0.0962 - accuracy: 0.9730
Epoch 9/10
179/179 [==============================] - 0s 2ms/step - loss: 0.0712 - accuracy: 0.9819
Epoch 10/10
179/179 [==============================] - 0s 2ms/step - loss: 0.0578 - accuracy: 0.984

## 루브릭 평가문항 1. 이미지 분류기 모델이 성공적으로 만들어졌는가?

# >훈련결과 : 정확도 98%

# 테스트 데이터 적재

In [89]:
import numpy as np
import os, glob

def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=643   
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("테스트데이터(x_test)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/mini_projects/rps/test/"

(x_test, y_test)=load_data(image_dir_path)

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

테스트데이터(x_test)의 이미지 개수는 642 입니다.
x_test shape: (643, 28, 28, 3)
y_test shape: (643,)


## 테스트 데이터 normalization

In [90]:
import numpy as np
print('최소값:',np.min(x_test), ' 최대값:',np.max(x_test))

x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

최소값: 0  최대값: 255


## 테스트 데이터로 모델 평가

In [91]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

test_loss, test_accuracy = model.evaluate(x_test_norm,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

21/21 - 2s - loss: 0.1270 - accuracy: 0.9533
test_loss: 0.1270076036453247 
test_accuracy: 0.9533436894416809


# >테스트 정확도: 95%

## 모델 개선

### 하이퍼 파라미터 조정

In [92]:
n_channel_1=32
n_channel_2=128
n_dense=128
n_train_epoch=15

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

# 모델 재훈련
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])


model.fit(x_train_norm, y_train, epochs=n_train_epoch)



Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 11, 11, 128)       36992     
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 5, 5, 128)         0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 3200)              0         
_________________________________________________________________
dense_42 (Dense)             (None, 128)               409728    
_________________________________________________________________
dense_43 (Dense)             (None, 3)               

## 루브릭 평가문항 3. 분류모델의 test accuracy가 기준 (60%) 이상 높게 나왔는가? 

# >>최종결과 : 정확도 99%

In [93]:
# 모델 시험
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

21/21 - 2s - loss: 0.0408 - accuracy: 0.9907
test_loss: 0.04082171246409416 
test_accuracy: 0.9906687140464783


# 프로젝트 일지
2020년 1월 5일과 6일 이틀에 걸쳐 작업을 했다.   
   
   
__1. 1차 시기 (실은 약 200여회차...)__   
   
훈련 데이터셋 3000장, 테스트 데이터셋 300장 으로 모델을 평가했다.   
학습시 정확도는 90%가 넘었지만 테스트 정확도는 40%대에 머물렀다.   
Conv2D 인자와 Dense 인자, 총 3개의 하이퍼 파라미터를   
두자리 수부터 네자리 수까지 가능한 모든 조합으로 섞어   
200여회 남짓 모델을 돌려보았지만   
테스트 정확도가 60%를 넘지 못했다.   
   
__2. 1차 시기의 실패 원인__   
   
너무 빡친 나머지 팀 슬랙에 여쭤보니   
훈련 데이터셋과 테스트 데이터셋에 쓴 이미지들을 선별하지 않고 그냥 사용한 점이   
문제인 것 같다는 지적을 받았다.   
같은 팀원 분의 조언대로 가위, 바위, 보 이미지들을 육안으로 보아도 분명히 식별가능한 데이터로 분류하여   
각각 훈련 데이터셋과, 테스트데이터셋에 나누어 분류하니   
테스트 이미지는 218장, 훈련 이미지는 2301장만 남아   
약 30%의 데이터 손실이 일어났다.   
하지만 이렇게 했을 때도 최종 테스트 정확도가 50%를 넘지 못했다.   
   
__3. 2차 시기__   
   
훈련 데이터 셋의 크기는 그대로 두고   
테스트 데이터 셋의 크기를 2배 늘려 425장을 사용했다.   
그 결과 테스트 정확도가 70%까지 나왔다.  
   
__4. 3차 시기__   
   
훈련 데이터셋의 크기를 2배로 늘려 4050장을 사용하고   
테스트 데이터셋은 원래대로 218장을 사용했더니   
최종 테스트 정확도가 100%까지 나왔다.   
하지만 3차 시기의 경우 학습시 정확도가 97%였던 것에 비해   
최초 모델 평가에서 99%가 나와   
오버 피팅이 의심되었다. 하지만 원인은 알 수 없었다. (퍼실님 왜 이렇게 되는 건지 알려 주실 수 있을까요?)   
   
__5. 4차 시기__   
   
훈련 데이터셋을 5702장, 테스트 데이터셋을 643장 사용하여 돌려본 결과  
학습시 정확도는 98%   
최초 테스트 정확도는 96%   
하이퍼 파라미터 조정결과 99%가 나왔다.   






# 회고   
   

__데이터의 질이 중요하다는 걸 깨달았을 때__   
처음엔 너무나 실망스러웠다.   
훈련데이터와 테스트데이터를 나눌 때, 테스트 데이터를 unseen data로 남겨둬야 하는데 그렇게 해선   
결코 원하는 결과가 나오지 않는다는 사실이 너무나 끔찍했다.   
하지만 마음을 가라 앉히고 내가 이 모델에서 무엇을 기대해야 하는 지를 생각해보았다.
1. 이미지의 퀄리티가 너무 낮기 때문에
2. 다양한 각도에서 촬영한 가위, 바위, 보 이미지 대신 2차원 상에서 분명히 구분가능한 이미지 들로만 데이터를 정리한 뒤   
3. 훈련데이터와 테스트데이터를 비슷한 수준의 이미지로 분류하여 모델을 돌림으로써   
4. 이 프로젝트에서 기대할 수 있는 것은 문자 그대로 명백한 가위, 바위, 보 이미지를 분류해내는 것이 최선이다.   

__나아가 이 프로젝트에서 내가 무엇을 배웠는지를 생각해보았다.__   
1. 데이터의 전처리는 단순히 알고리즘에 넣을 수 있게 데이터의 수치를 조정하는 것만이 아니라   
   데이터의 질을 평균 이상으로 맞춰주는 작업도 포함된다.   
2. 데이터의 질도 중요하지만, 그 전에 내가 만들고자 하는 모델이 어느 정도 수준의 결과를 낼 수 있는 모델인지를 먼저 분명히해야 한다.   
3. 모델의 성능과 데이터의 수준을 정확히 파악했다 하더라도, 모델의 성능을 개선시키는 방법을 알지 못하면 1회용 모델을 만들게 될 뿐이다.   
4. 데이터의 질 문제를 해결해도, 절대적인 데이터 양이 적으면 좋은 성능을 기대하기 어렵다.   
5. 나의 지식 수준이 한참 낮아 모델이 정말 어떤 원리로 작동하는지 모르는 상태에서는 절대로 결과에 욕심을 내서도 짜증을 내서도 안된다.   
6. 컴퓨터도 사용자가 너무 무식하면 두손 놓아버린다. (ResourceExhaustedError가 한 번 뜨니 더이상 코드런이 안돼 컴퓨터를 껐다 켜야 했다)   
   
__반성할 점__
1. 오늘 하루 종일 뇌가 가위바위보에만 쏠려버려 민채퍼실님의 코딩마스터 수업을 거의 못들었다..   
2. 1차 시기에서 그토록 많은 에너지와 시간을 낭비하지 않을 수 있었다. 가만히 생각해 보면 이미지 퀄리티를 고려해야한다는 점이   
   노드에 설명돼 있었다. 머리를 쓰자...   
   
__더 공부해야할 부분__   
함수가 길어지면 변수가 인자가 되고, 그 변수가 다시 다른 함수의 인자가 되고, 꼬리에 꼬리를 물면서 전체 그림을 놓친다.   
기본적인 문자열 함수들에 대한 이해가 부족하다. 슬라이싱, 인덱싱 기법들...   
넘파이 배열에 너무 약하다.   
딥러닝 용어를 더 공부해야겠다.